In [1]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [5]:
file_path = r"C:\Users\Kalpesh\Downloads\Jayesh Testing\Final_Output\Book1_new.xlsx"  
building_tower_df = pd.read_excel(file_path, sheet_name="Building_Tower")
coordinates_df = pd.read_excel(file_path, sheet_name="Coordinates")

In [6]:
tower_faces_data = []

for (building, tower), group in coordinates_df.groupby(["Building", "Tower"]):    

    x = group['X'].tolist()
    y = group['Y'].tolist()
    z = group['Z'].tolist()
    
    for i in range(4):
        face = i + 1
        #print(face)
        
        point1 = [x[i], y[i], z[i]]
        point2 = [x[(i + 1)%4], y[(i + 1)%4], z[(i + 1)%4]]             # Wrap around to the first point after the last
        #print([point1, point2])

        tower_face_id = f"{building}-{tower}-{face}"
        point1_mod = f"{tower_face_id}-TC{i+1}"
        point2_mod = f"{tower_face_id}-TC{((i+1)%4)+1}"                 #TC5 becomes TC1
        #print([point1_mod, point2_mod])

        tower_faces_data.append({
            "Building number": building, "Tower number": tower, "Face": face, "Tower_face_id": tower_face_id, "Point1": point1, "Point2": point2, "Point1_mod": point1_mod, "Point2_mod": point2_mod
        })

tower_faces_df = pd.DataFrame(tower_faces_data)

with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    tower_faces_df.to_excel(writer, sheet_name="Tower_Faces", index=False)

print("Tower_Faces sheet Check")

Tower_Faces sheet Check


In [7]:
building_faces_data = []

for building, group in coordinates_df.groupby("Building"):
    # Filter for valid vertices
    valid_vertices = group[group['Building_vertices_identifier'].isin(['BC1', 'BC2', 'BC3', 'BC4'])]
    
    # Sort the vertices in sequential order based on the identifier
    valid_vertices = valid_vertices.sort_values(by="Building_vertices_identifier", key=lambda x: x.str.extract(r'(\d+)$')[0].astype(int))
    
    # Extract coordinates
    x = valid_vertices['X'].tolist()
    y = valid_vertices['Y'].tolist()
    z = valid_vertices['Z'].tolist()

    for i in range(4):
        face = i + 1
        
        # Define points to form the face
        point1 = [x[i], y[i], z[i]]
        point2 = [x[(i + 1) % 4], y[(i + 1) % 4], z[(i + 1) % 4]]  # Wrap around to close the loop
        
        building_face_id = f"{building}-{face}"
        point1_mod = f"{building}-BC{i+1}"
        point2_mod = f"{building}-BC{((i+1)%4)+1}"  # (BC5 becomes BC1)
        
        building_faces_data.append({
            "Building number": building,
            "Face": face,
            "Building_Face_id": building_face_id,
            "Point1": point1,
            "Point2": point2,
            "Point1_mod": point1_mod,
            "Point2_mod": point2_mod
        })

# Create DataFrame
building_faces_df = pd.DataFrame(building_faces_data)

# Write to Excel file
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    building_faces_df.to_excel(writer, sheet_name="Building_Faces", index=False)

print("Building_Faces sheet Check")

Building_Faces sheet Check


In [8]:
observer_obstacle_data = []                                                                                         #root form of dataframe

for observer_idx, observer_row in building_tower_df.iterrows():                                                     #select observer tower from Building_tower sheet
    observer_building = observer_row["Building"]
    observer_tower = observer_row["Tower"]
    
    for obstacle_idx, obstacle_row in building_tower_df.iterrows():                                                 #select obstacle tower from Building_tower sheet (may or may not be same as observer tower)
        obstacle_building = obstacle_row["Building"]
        obstacle_tower = obstacle_row["Tower"]
        
        if observer_building == obstacle_building:                                                                  #if satisfied, take coordinates from Tower_Faces
            relevant_obsv_faces_df = tower_faces_df[(tower_faces_df["Building number"] == observer_building) &
                                               (tower_faces_df["Tower number"] == observer_tower)]
            relevant_obst_faces_df = tower_faces_df[(tower_faces_df["Building number"] == obstacle_building) &
                                               (tower_faces_df["Tower number"] == obstacle_tower)]
            # print('Observer_SameB: f{relevant_obsv_faces_df}')
            # print('Obstacle_SameB: f{relevant_obst_faces_df}')
        else:                                                                                                       #else, take coordinates from Building_Faces
            relevant_obsv_faces_df = building_faces_df[(building_faces_df["Building number"] == observer_building)]
            relevant_obst_faces_df = building_faces_df[(building_faces_df["Building number"] == obstacle_building)]
            # print('Observer_DiffB: f{relevant_obsv_faces_df}')
            # print('Obstacle_DiffB: f{relevant_obst_faces_df}')

        for obs_face_idx, observer_face_row in relevant_obsv_faces_df.iterrows():
            for obs_face2_idx, obstacle_face_row in relevant_obst_faces_df.iterrows():
                if observer_face_row["Face"] != obstacle_face_row["Face"]:                                          #no overlapping of observer and obstacle wall
                    observer_obstacle_data.append({
                        "Observer_Building": observer_building, "Observer_Tower": observer_tower, "Observer_Face": observer_face_row["Face"], "Observer_Point1": observer_face_row["Point1"], "Observer_Point2": observer_face_row["Point2"], "Obstacle_Building": obstacle_building, "Obstacle_Tower": obstacle_tower, "Obstacle_Face": obstacle_face_row["Face"], "Obstacle_Point1": obstacle_face_row["Point1"], "Obstacle_Point2": obstacle_face_row["Point2"], "Obstacle_Point1_mod": obstacle_face_row["Point1_mod"], "Obstacle_Point2_mod": obstacle_face_row["Point2_mod"],  
                    })

observer_obstacle_df = pd.DataFrame(observer_obstacle_data)

In [9]:
observer_point3_list = []
observer_point4_list = []
obstacle_point3_list = []
obstacle_point4_list = []

for idx, row in observer_obstacle_df.iterrows():
    observer_point1 = row["Observer_Point1"]
    observer_point2 = row["Observer_Point2"]
    observer_point3 = (observer_point1[0], observer_point1[1], 300)
    observer_point4 = (observer_point2[0], observer_point2[1], 300)
    
    obstacle_point1 = row["Obstacle_Point1"]
    obstacle_point2 = row["Obstacle_Point2"]
    obstacle_point3 = (obstacle_point1[0], obstacle_point1[1], 300)
    obstacle_point4 = (obstacle_point2[0], obstacle_point2[1], 300)
    
    observer_point3_list.append(observer_point3)
    observer_point4_list.append(observer_point4)
    obstacle_point3_list.append(obstacle_point3)
    obstacle_point4_list.append(obstacle_point4)

observer_obstacle_df["Observer_Point3"] = observer_point3_list
observer_obstacle_df["Observer_Point4"] = observer_point4_list
observer_obstacle_df["Obstacle_Point3"] = obstacle_point3_list
observer_obstacle_df["Obstacle_Point4"] = obstacle_point4_list

with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    observer_obstacle_df.to_excel(writer, sheet_name="Observer_Obstacle", index=False)

In [10]:
# observer_obstacle_data = []                                                     #root form of dataframe

# for observer_index, observer_row in tower_faces_df.iterrows():
#     observer_building = observer_row['Building number']
#     observer_face_id = observer_row['Tower_face_id']
#     observer_point1 = observer_row['Point1']
#     observer_point2 = observer_row['Point2']

#     for obstacle_index, obstacle_row in tower_faces_df.iterrows():
#         if observer_index != obstacle_index:                                    #for different observer and obstacle faces
#             obstacle_building = obstacle_row['Building number']
#             if observer_building == obstacle_building:                          #if obsever face is from same building as obstacle face
#                 obstacle_point1 = obstacle_row['Point1']                        #take point1 and point2 from Tower_Faces (matching building number of obstacle face)
#                 obstacle_point2 = obstacle_row['Point2']
#                 #print([obstacle_point1, obstacle_point2])

#                 obstacle_point1_mod = f"{obstacle_building}-{obstacle_row['Point1_mod']}"
#                 obstacle_point2_mod = f"{obstacle_building}-{obstacle_row['Point2_mod']}"
#                 #print([obstacle_point1_mod, obstacle_point2_mod])   

#             else:                                                               #if obstacle face is from different building than observer face
#                 matching_face = building_faces_df[     
#                     (building_faces_df['Building number'] == obstacle_building) & 
#                     (building_faces_df['Face'] == obstacle_row['Face'])
#                 ]                                                               #then take point1 and point2 from Building_Faces (matching building number of obstacle face)
#                 print(matching_face)                                                   
#                 obstacle_point1 = matching_face['Point1'].values[0]             #matching_face['column_name'] has other attributes while calling, hence we take only first attribute which is coordinates of the point
#                 obstacle_point2 = matching_face['Point2'].values[0]
#                 #print([obstacle_point1, obstacle_point2])

#                 obstacle_point1_mod = f"{obstacle_building}-{matching_face['Point1_mod'].values[0]}"
#                 obstacle_point2_mod = f"{obstacle_building}-{matching_face['Point2_mod'].values[0]}"
#                 #print([obstacle_point1_mod, obstacle_point2_mod])   

#             observer_obstacle_data.append({
#                 "Observer_Face": observer_face_id, "Observer_Point1": observer_point1, "Observer_Point2": observer_point2, "Obstacle_Face": obstacle_row['Tower_face_id'], "Obstacle_Point1_mod": obstacle_point1_mod, "Obstacle_Point1": obstacle_point1, "Obstacle_Point2_mod": obstacle_point2_mod, "Obstacle_Point2": obstacle_point2
#             })

# observer_obstacle_df = pd.DataFrame(observer_obstacle_data)

# # # Convert list columns to tuples to make them hashable
# # observer_obstacle_df['Observer_Point1'] = observer_obstacle_df['Observer_Point1'].apply(tuple)
# # observer_obstacle_df['Observer_Point2'] = observer_obstacle_df['Observer_Point2'].apply(tuple)
# # observer_obstacle_df['Obstacle_Point1'] = observer_obstacle_df['Obstacle_Point1'].apply(tuple)
# # observer_obstacle_df['Obstacle_Point2'] = observer_obstacle_df['Obstacle_Point2'].apply(tuple)

# # # Now, drop duplicates based on the unique combinations of the four columns
# # observer_obstacle_unique_df = observer_obstacle_df.drop_duplicates(subset=['Observer_Point1', 'Observer_Point2', 'Obstacle_Point1', 'Obstacle_Point2'])

# with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
#     observer_obstacle_df.to_excel(writer, sheet_name="Observer_Obstacle", index=False)

# print("Observer_Obstacle sheet Check")

In [11]:
import datetime  

latitude = 28.4595                                          # Latitude for the location (New Delhi, India)
longitude = 77.0266                                         # Longitude for the location (New Delhi, India)

time = datetime.datetime(2023, 7, 15, 14, 0, 0)             # July 15, 2023, at 14:00 (2 PM)

sun_azimuth_morning = 67.7756271507185                      # Sun azimuth in the morning
sun_azimuth_afternoon = 228.2852958                         # Sun azimuth at 1 PM
sun_azimuth_evening = 309.08750082861                       # Sun azimuth in the evening

sun_elevation_morning = 4.557596553                         # Sun elevation in the morning
sun_elevation_afternoon = 79.96185006                       # Sun elevation at 1 PM
sun_elevation_evening = -19.23511277                        # Sun elevation in the evening (below the horizon)

In [ ]:
def wall_orientation(cross_product1, cross_product2):
    if cross_product1 < 0 and cross_product2 < 0:
        return "Back"  
    elif cross_product1 > 0 and cross_product2 > 0:
        return "Front"  
    else:
        return "Parallel"  

def calculate_cross_product(observer_face, building_face):
    observer_point1, observer_point2 = [point for point in observer_face[:4] if point[2] == 0]
    
    building_point1, building_point2 = [point for point in building_face[:4] if point[2] == 0]

    ab = np.array(observer_point2) - np.array(observer_point1)
    w1 = np.array(building_point1) - np.array(observer_point1)  # Vector to the first building point
    w2 = np.array(building_point2) - np.array(observer_point1)  # Vector to the second building point
    
    cross_product_result = np.cross(ab, w1).astype(int)  # Cross product with first building vector
    cross_product_result1 = np.cross(ab, w2).astype(int)  # Cross product with second building vector
    
    return cross_product_result, cross_product_result1  

def check_azimuth_overlap(min_obstacle, max_obstacle, range_start, range_end):
    # Handle cases where the range wraps around 360 degrees
    if range_start > range_end:
        return not (max_obstacle < range_start and min_obstacle > range_end)  # Check for overlap
    return not (min_obstacle > range_end or max_obstacle < range_start)  # Check for overlap in standard range

def check_buildings(building_x1, building_y1, building_x2, building_y2, observer_x, observer_y, sun_azimuth_morning, sun_azimuth_evening):
    # Calculate the azimuth angles of the building points relative to the observer
    azimuth_obstacle1 = np.degrees(np.arctan2(building_x1 - observer_x, building_y1 - observer_y)) % 360  
    azimuth_obstacle2 = np.degrees(np.arctan2(building_x2 - observer_x, building_y2 - observer_y)) % 360   
    
    ab_aw1_z = building_x1 - observer_x  
    ab_aw2_z = building_x2 - observer_x  # Cross product = [obst_x - obsv_x]*1 - [obst_y - obsv_y]*0
    
    orientation = wall_orientation(ab_aw1_z, ab_aw2_z)
    sun_range_crosses_zero = sun_azimuth_morning > sun_azimuth_evening  # Check if the sun path crosses zero degrees
    
    # Determine minimum and maximum azimuth angles of the obstacles
    min_obstacle = min(azimuth_obstacle1, azimuth_obstacle2)
    max_obstacle = max(azimuth_obstacle1, azimuth_obstacle2)
    
    if orientation in ["Front", "Back"]:  # If the wall is in front or back of the observer
        if sun_range_crosses_zero:  # If the sun path wraps around
            morning_range = (sun_azimuth_morning, 360)  # Morning azimuth range
            evening_range = (0, sun_azimuth_evening)  # Evening azimuth range
            # Check for overlaps with both morning and evening ranges
            overlaps_morning = check_azimuth_overlap(min_obstacle, max_obstacle, *morning_range)
            overlaps_evening = check_azimuth_overlap(min_obstacle, max_obstacle, *evening_range)
            return (overlaps_morning or overlaps_evening)  #Return true if any overlap is found
        else:
            # Check for overlap in the normal azimuth range
            return check_azimuth_overlap(min_obstacle, max_obstacle, sun_azimuth_morning, sun_azimuth_evening)
    else:  # If the wall is parallel to the observer's view
        # Define a function to check if a point could cast a shadow based on azimuth
        def point_could_cast_shadow(azimuth):
            return (azimuth > sun_azimuth_morning and azimuth < sun_azimuth_evening) if not sun_range_crosses_zero else \
                   (azimuth > sun_azimuth_morning or azimuth < sun_azimuth_evening)

        # Check if either building point could cast a shadow
        return (point_could_cast_shadow(azimuth_obstacle1) or point_could_cast_shadow(azimuth_obstacle2))


In [ ]:
def plot_shadow_analysis(observer_wall1_x, observer_wall1_y, observer_wall2_x, observer_wall2_y,
                         building_wall1_x, building_wall1_y, building_wall2_x, building_wall2_y,
                         sun_azimuth_morning, sun_azimuth_afternoon, sun_azimuth_evening,
                         observer_tower, observer_face, obstacle_tower, obstacle_face, obstacle_building):
    plt.figure(figsize=(12, 10))
    
    plt.plot([observer_wall1_x, observer_wall2_x], [observer_wall1_y, observer_wall2_y], 
             marker='o', color='black', linewidth=2, label='Observer Wall')
    
    plt.annotate(f'({observer_wall1_x}, {observer_wall1_y})', (observer_wall1_x, observer_wall1_y), 
                 xytext=(5, 5), textcoords='offset points', fontsize=8, 
                 bbox=dict(facecolor='white', edgecolor='none', alpha=0.7))
    plt.annotate(f'({observer_wall2_x}, {observer_wall2_y})', (observer_wall2_x, observer_wall2_y), 
                 xytext=(5, 5), textcoords='offset points', fontsize=8, 
                 bbox=dict(facecolor='white', edgecolor='none', alpha=0.7))
    
    plt.plot([building_wall1_x, building_wall2_x], [building_wall1_y, building_wall2_y], 
             color='blue', linewidth=2, marker='o', label='Building Wall')
    
    shadow_info1 = check_buildings(building_wall1_x, building_wall1_y, building_wall2_x, building_wall2_y, 
                                   observer_wall1_x, observer_wall1_y, sun_azimuth_morning, sun_azimuth_evening)
    shadow_info2 = check_buildings(building_wall1_x, building_wall1_y, building_wall2_x, building_wall2_y, 
                                   observer_wall2_x, observer_wall2_y, sun_azimuth_morning, sun_azimuth_evening)
    
    if ((shadow_info1 and not shadow_info2) or (shadow_info2 and not shadow_info1)):
        status_text = "Partial"
        color = 'orange'
    elif (shadow_info1 and shadow_info2):
        status_text = "Full Shadow"
        color = 'red'
    else:
        status_text = 'No Shadow'
        color = 'green'

    midpoint_x = (observer_wall1_x + observer_wall2_x) / 2
    midpoint_y = (observer_wall1_y + observer_wall2_y) / 2
    plt.text(midpoint_x, midpoint_y, status_text, ha='center', va='center', bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'), color=color, fontsize=10)

    # Calculate unit vector for the sun azimuth (same for both points)
    sun_vector_morning_x = np.sin(np.radians(sun_azimuth_morning))
    sun_vector_morning_y = np.cos(np.radians(sun_azimuth_morning))

    # Add quivers for morning azimuth (parallel arrows)
    plt.quiver(observer_wall1_x, observer_wall1_y, sun_vector_morning_x, sun_vector_morning_y,
            scale=10, color='#f96343', label=f'Sun Azimuth Morning : {sun_azimuth_morning:.2f}°', angles='xy')
    plt.quiver(observer_wall2_x, observer_wall2_y, sun_vector_morning_x, sun_vector_morning_y,
            scale=10, color='#f96343')

    # Afternoon azimuth
    sun_vector_afternoon_x = np.sin(np.radians(sun_azimuth_afternoon))
    sun_vector_afternoon_y = np.cos(np.radians(sun_azimuth_afternoon))

    plt.quiver(observer_wall1_x, observer_wall1_y, sun_vector_afternoon_x, sun_vector_afternoon_y,
            scale=10, color='#d3de50', label=f'Sun Azimuth Afternoon : {sun_azimuth_afternoon:.2f}°', angles='xy')
    plt.quiver(observer_wall2_x, observer_wall2_y, sun_vector_afternoon_x, sun_vector_afternoon_y,
            scale=10, color='#d3de50')

    # Evening azimuth
    sun_vector_evening_x = np.sin(np.radians(sun_azimuth_evening))
    sun_vector_evening_y = np.cos(np.radians(sun_azimuth_evening))

    plt.quiver(observer_wall1_x, observer_wall1_y, sun_vector_evening_x, sun_vector_evening_y,
            scale=10, color='#df684e', label=f'Sun Azimuth Evening : {sun_azimuth_evening:.2f}°', angles='xy')
    plt.quiver(observer_wall2_x, observer_wall2_y, sun_vector_evening_x, sun_vector_evening_y,
            scale=10, color='#df684e')

    
    plt.text(0.02, 0.98, f'Sun Path:\nMorning: {sun_azimuth_morning:.1f}°\nAfternoon: {sun_azimuth_afternoon:.1f}°\nEvening: {sun_azimuth_evening:.1f}°',
             transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.7), verticalalignment='top')
    
    plt.title('Shadow Analysis for Observer Wall')
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.legend(loc='upper left')
    plt.grid(True)
    plt.tight_layout()
    
    filename = f"{observer_tower}_{observer_face}_{obstacle_building}_{obstacle_tower}_{obstacle_face}"
    save_path = fr'C:\Users\Kalpesh\Downloads\Jayesh Testing\Final_Output\Plots\North_wall_new\{filename}.png'
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
def plot_wall_orientations(observer_wall1_x, observer_wall1_y, observer_wall2_x, observer_wall2_y,
                           building_wall1_x, building_wall1_y, building_wall2_x, building_wall2_y,
                           observer_building,observer_tower, observer_face, obstacle_tower, obstacle_face, obstacle_building):
    plt.figure(figsize=(12, 10))
    
    # Filter the faces for the observer building
    building_faces = building_faces_df[building_faces_df['Building number'] == observer_building]

    # Draw the entire observer building in light color
    for _, face in building_faces.iterrows():
        point1 = face['Point1']
        point2 = face['Point2']
        plt.plot([point1[0], point2[0]], [point1[1], point2[1]], color='lightgreen', linewidth=1)
    
    # Highlight the observer face in consideration
    observer_face_coords = building_faces[building_faces['Face'] == observer_face]
    if not observer_face_coords.empty:
        point1 = observer_face_coords.iloc[0]['Point1']
        point2 = observer_face_coords.iloc[0]['Point2']
        plt.plot([point1[0], point2[0]], [point1[1], point2[1]], color='black', linewidth=2, label='Observer Face')
        plt.annotate(f"Face {observer_face}", ((point1[0] + point2[0]) / 2, (point1[1] + point2[1]) / 2), 
                     textcoords="offset points", xytext=(5, 5), fontsize=8, bbox=dict(facecolor='white', edgecolor='none', alpha=0.7))

    # Plot the obstacle wall
    plt.plot([building_wall1_x, building_wall2_x], [building_wall1_y, building_wall2_y], color='blue', linewidth=2, marker='o', label='Obstacle Wall')
    
    # Annotate obstacle wall points
    plt.annotate(f'({building_wall1_x}, {building_wall1_y})', (building_wall1_x, building_wall1_y), xytext=(5, 5), textcoords='offset points', fontsize=8, bbox=dict(facecolor='white', edgecolor='none', alpha=0.7))
    plt.annotate(f'({building_wall2_x}, {building_wall2_y})', (building_wall2_x, building_wall2_y), xytext=(5, 5), textcoords='offset points', fontsize=8, bbox=dict(facecolor='white', edgecolor='none', alpha=0.7))
    
    plt.xlabel('West-East')
    plt.ylabel('South-North')
    plt.title(f'Building {observer_building} with Highlighted Observer Face {observer_face}')
    plt.grid(True)
    plt.legend(loc='lower right')
    
    # Save plot with a unique filename
    filename = f"{observer_tower}_{observer_face}_{obstacle_building}_{obstacle_tower}_{obstacle_face}"
    save_path = fr'C:\Users\Kalpesh\Downloads\Jayesh Testing\Final_Output\Plots\Wall_Orientation_new\{filename}'
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()


In [14]:
import math
import pytz
import pysolar.solar as ps

def calculate_Ow(observer_wall_x, observer_wall_y, observer_wall_z, building_wall_x, building_wall_y, building_wall_z):
    """Calculate the angle of obstruction (Ow) between the observer wall and building wall."""
    
    # Calculate the squared distance between the observer and building wall
    squared_distance = (building_wall_y - observer_wall_y) ** 2 + (building_wall_x - observer_wall_x) ** 2
    if squared_distance < 0:
        raise ValueError("Error: Squared distance is negative.")
    
    distance = math.sqrt(squared_distance)  # Calculate Euclidean distance

    # Calculate the height difference between the two walls
    height_diff = building_wall_z - observer_wall_z

    # Calculate the angle Ow (angle of obstruction)
    Ow = 0 if distance == 0 else math.degrees(math.atan(height_diff / distance))
    
    return Ow

def get_sun_data(latitude, longitude, date_time):
    """Retrieve sun elevation and azimuth based on the given latitude, longitude, and time."""
    
    # Set timezone for the specified location
    timezone = pytz.timezone('Asia/Kolkata')
    date_time_local = timezone.localize(date_time)  # Localize the datetime to the specified timezone

    # Calculate sun position using pysolar
    sun_elevation = ps.get_altitude(latitude, longitude, date_time_local)
    sun_azimuth = ps.get_azimuth(latitude, longitude, date_time_local)

    return sun_elevation, sun_azimuth

def get_shadow_status_and_elevation(observer_wall1_x, observer_wall1_y, observer_wall1_z, observer_wall2_x, observer_wall2_y, observer_wall2_z, 
                                     building_wall1_x, building_wall1_y, building_wall1_z, building_wall2_x, building_wall2_y, building_wall2_z, 
                                     latitude, longitude, time):
    """Determine shadow status for two observer walls based on building walls and sun position."""
    
    # Calculate Ow for the two observer walls against the two building walls
    obs1_bldg1 = calculate_Ow(observer_wall1_x, observer_wall1_y, observer_wall1_z, building_wall1_x, building_wall1_y, building_wall1_z)
    obs1_bldg2 = calculate_Ow(observer_wall1_x, observer_wall1_y, observer_wall1_z, building_wall2_x, building_wall2_y, building_wall2_z)
    obs2_bldg1 = calculate_Ow(observer_wall2_x, observer_wall2_y, observer_wall2_z, building_wall1_x, building_wall1_y, building_wall1_z)
    obs2_bldg2 = calculate_Ow(observer_wall2_x, observer_wall2_y, observer_wall2_z, building_wall2_x, building_wall2_y, building_wall2_z)

    # Get sun data for the current time
    elevation, azimuth = get_sun_data(latitude, longitude, time)

    # Determine shadow status based on the elevation of the sun relative to the maximum Ow
    shadow_status_obs1 = elevation <= max(obs1_bldg1, obs1_bldg2)
    shadow_status_obs2 = elevation <= max(obs2_bldg1, obs2_bldg2)

    return shadow_status_obs1, shadow_status_obs2, elevation, obs1_bldg1, obs1_bldg2, obs2_bldg1, obs2_bldg2

def get_closest_times(elevation, excel_file):
    """Find the closest times in an Excel file to a given sun elevation."""
    
    # Read data from Excel file
    df = pd.read_excel(excel_file)
    df['Time'] = pd.to_datetime(df['Time'])  # Convert time column to datetime

    # Split DataFrame into two based on time (before and after noon)
    df_before_12 = df[df['Time'].dt.hour < 12]
    df_after_12 = df[df['Time'].dt.hour >= 12]

    # Find closest elevation times before noon
    closest_elevation_row_before_12 = df_before_12.iloc[(df_before_12['Elevation'] - elevation).abs().argsort()[:1]]
    closest_time_before_12 = pd.Timestamp(closest_elevation_row_before_12['Time'].values[0]).strftime('%H')

    # Find closest elevation times after noon
    closest_elevation_row_after_12 = df_after_12.iloc[(df_after_12['Elevation'] - elevation).abs().argsort()[:1]]
    closest_time_after_12 = pd.Timestamp(closest_elevation_row_after_12['Time'].values[0]).strftime('%H')

    return int(closest_time_before_12), int(closest_time_after_12)

In [ ]:
import os

def calculate_azimuth_ccw_or_cw(x_observer, y_observer, x_target, y_target, is_ccw=False):
    if (x_target == x_observer):
        return 0
    angle = np.degrees(np.arctan2(y_target - y_observer, x_target - x_observer)) % 360
    if is_ccw:
        return -angle + 90
    return 90 - angle

def get_shadow_status_and_azimuth(obs_x1, obs_y1, obs_x2, obs_y2, 
                                 building_x1, building_y1, building_x2, building_y2,
                                 latitude, longitude, time):
    
    # Calculate angles for observer 1
    t1_angle = calculate_azimuth_ccw_or_cw(obs_x1, obs_y1, building_x1, building_y1)
    t2_angle = calculate_azimuth_ccw_or_cw(obs_x1, obs_y1, building_x2, building_y2)
    
    # Calculate angles for observer 2
    t3_angle = calculate_azimuth_ccw_or_cw(obs_x2, obs_y2, building_x1, building_y1)
    t4_angle = calculate_azimuth_ccw_or_cw(obs_x2, obs_y2, building_x2, building_y2)
    
    elevation, azimuth = get_sun_data(latitude, longitude, time)
    
    cross_prod_obs1_b1 = building_x1 - obs_x1               # Cross product = [obst_x - obsv_x]*1 - [obst_y - obsv_y]*0
    cross_prod_obs1_b2 = building_x2 - obs_x1               
    cross_prod_obs2_b1 = building_x1 - obs_x2 
    cross_prod_obs2_b2 = building_x2 - obs_x2 
    
    shadow_status_1 = False
    if (cross_prod_obs1_b1 * cross_prod_obs1_b2 < 0):
        if t1_angle <= azimuth <= t2_angle or t1_angle >= azimuth >= t2_angle:
            shadow_status_1 = True
            
    shadow_status_2 = False
    if (cross_prod_obs2_b1 * cross_prod_obs2_b2 < 0):
        if t3_angle <= azimuth <= t4_angle or t3_angle >= azimuth >= t4_angle:
            shadow_status_2 = True
    
    return shadow_status_1, shadow_status_2, azimuth, t1_angle, t2_angle, t3_angle, t4_angle

def get_time_from_azimuth(azimuth, excel_file):
    data = pd.read_excel(excel_file)
    data['Time'] = pd.to_datetime(data["Time"])

    closest_row = min(data.iterrows(), key=lambda x: abs(x[1]['Azimuth'] - azimuth))[1]
    return closest_row['Time'].hour

In [16]:
observer_walls = []
building_walls = []
observer_building_combine = []

solar_file = r"C:\Users\Kalpesh\Downloads\NewApproach Sun Exposure\NewApproach Sun Exposure\Azimuth_elevation\Sun_data_Pysolar_new.xlsx"

for idx, row in observer_obstacle_df.iterrows():
    observer_wall = [
        (row['Observer_Point1'][0], row['Observer_Point1'][1], row['Observer_Point1'][2]),
        (row['Observer_Point3'][0], row['Observer_Point3'][1], row['Observer_Point3'][2]),
        (row['Observer_Point2'][0], row['Observer_Point2'][1], row['Observer_Point2'][2]),
        (row['Observer_Point4'][0], row['Observer_Point4'][1], row['Observer_Point4'][2]),
        row['Observer_Building'], row['Observer_Tower'], row['Observer_Face']
    ]
    
    building_wall = [
        (row['Obstacle_Point1'][0], row['Obstacle_Point1'][1], row['Obstacle_Point1'][2]),
        (row['Obstacle_Point3'][0], row['Obstacle_Point3'][1], row['Obstacle_Point3'][2]),
        (row['Obstacle_Point2'][0], row['Obstacle_Point2'][1], row['Obstacle_Point2'][2]),
        (row['Obstacle_Point4'][0], row['Obstacle_Point4'][1], row['Obstacle_Point4'][2]),
        row['Obstacle_Tower'], row['Obstacle_Face'], row['Obstacle_Building']
    ]
    
    building_status = check_buildings(building_wall[0][0], building_wall[0][1], building_wall[2][0], building_wall[2][1], observer_wall[0][0], observer_wall[0][1], sun_azimuth_morning, sun_azimuth_evening) or check_buildings(building_wall[0][0], building_wall[0][1], building_wall[2][0], building_wall[2][1], observer_wall[2][0], observer_wall[2][1], sun_azimuth_morning, sun_azimuth_evening)
    
    #Plot sun_morning, sun_afternoon and sun_evening quivers in each plot
    # plot_shadow_analysis(observer_wall[0][0], observer_wall[0][1], observer_wall[2][0], observer_wall[2][1],
    #                      building_wall[0][0], building_wall[0][1], building_wall[2][0], building_wall[2][1],
    #                      sun_azimuth_morning, sun_azimuth_afternoon, sun_azimuth_evening, 
    #                      observer_wall[5], observer_wall[6], building_wall[4], building_wall[5], building_wall[6])
        
    cross_product_result = calculate_cross_product(observer_wall[:4], building_wall[:4])
    
    wall_orientation_status = wall_orientation(cross_product_result[0][2], cross_product_result[1][2])

    #Plot the observer tower as whole as well
    # plot_wall_orientations(observer_wall[0][0], observer_wall[0][1], observer_wall[2][0], observer_wall[2][1],
    #                      building_wall[0][0], building_wall[0][1], building_wall[2][0], building_wall[2][1],
    #                      observer_wall[4], observer_wall[5], observer_wall[6], building_wall[4], building_wall[5], building_wall[6])
            
    if(wall_orientation_status != 'Back' and building_status != False):
        ### ELEVATION FUNCTION CALCULATIONS
        shadow_status_obs1_elevation, shadow_status_obs2_elevation, elevation, t1_angle_elevation, t2_angle_elevation, t3_angle_elevation, t4_angle_elevation = get_shadow_status_and_elevation(observer_wall[0][0], observer_wall[0][1], observer_wall[0][2], observer_wall[2][0], observer_wall[2][1], observer_wall[2][2],
        building_wall[1][0], building_wall[1][1], building_wall[1][2], building_wall[3][0], building_wall[3][1], building_wall[3][2],
        latitude, longitude, time)
        #print([shadow_status_obs1_elevation, shadow_status_obs2_elevation])

        max_elev = max(t1_angle_elevation, t2_angle_elevation, t3_angle_elevation, t4_angle_elevation)

        closest_time_before_12, closest_time_after_12 = get_closest_times(max_elev, solar_file)

        ### AZIMUTH FUNCTION CALCULATION
        shadow_status_1_azimuth, shadow_status_2_azimuth, azimuth, t1_angle_azimuth, t2_angle_azimuth, t3_angle_azimuth, t4_angle_azimuth = get_shadow_status_and_azimuth(
            observer_wall[0][0], observer_wall[0][1], observer_wall[2][0], observer_wall[2][1], building_wall[1][0], building_wall[1][1], building_wall[3][0], building_wall[3][1], latitude, longitude, time)
        #print([shadow_status_1_azimuth, shadow_status_2_azimuth])

        min_azimuth = min(t1_angle_azimuth, t2_angle_azimuth, t3_angle_azimuth, t4_angle_azimuth)
        max_azimuth = max(t1_angle_azimuth, t2_angle_azimuth, t3_angle_azimuth, t4_angle_azimuth)

        min_azimuth_time = get_time_from_azimuth(min_azimuth, solar_file)
        max_azimuth_time = get_time_from_azimuth(max_azimuth, solar_file)

        # min_time should be lower bound of overlap of both azimuth and elevation times. If no overlap, return no shadow
        min_time = max(closest_time_before_12, min_azimuth_time)
        max_time = min(closest_time_after_12, max_azimuth_time)

    else:
        t1_angle_elevation, t2_angle_elevation, t3_angle_elevation, t4_angle_elevation, max_elev, closest_time_after_12, closest_time_before_12, t1_angle_azimuth, t2_angle_azimuth, t3_angle_azimuth, t4_angle_azimuth, min_azimuth, max_azimuth, min_azimuth_time, max_azimuth_time, min_time, max_time = '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'
        
    observer_building_combine.append([
        observer_wall[4],                               # Observer tower number
        observer_wall[5],                               # Observer wall number
        observer_wall[6],
        observer_wall[0:4],                             # Coordinates of the observer wall
        building_wall[6],                               # Building number
        building_wall[4],                               # Building tower number
        building_wall[5],                               # Building wall number
        building_wall[0:4],                             # Coordinates of the building wall
        building_status,                                # Status of shadow casting
        cross_product_result[0][2],                     # z-component of the first cross product
        cross_product_result[1][2],                     # z-component of the second cross product
        wall_orientation_status,
        t1_angle_elevation,
        t2_angle_elevation,
        t3_angle_elevation,
        t4_angle_elevation,
        max_elev,
        closest_time_before_12,
        closest_time_after_12,
        t1_angle_azimuth,
        t2_angle_azimuth,
        t3_angle_azimuth,
        t4_angle_azimuth,
        min_azimuth,
        max_azimuth,
        min_azimuth_time,
        max_azimuth_time,
        min_time,
        max_time
        ])

results_combine_df = pd.DataFrame(observer_building_combine, columns=['Tower_number', 'Observer_wall_no', 'Observer_Face' ,'observer_walls', 'Building No', 'tower_no', 'Building_wall_no', 'Building_walls', 'Building_status', 'ab*aw1_z', 'ab*aw2_z', 'wall_orientation', 't1_elevation', 't2_elevation', 't3_elevation', 't4_elevation', 'max_elevation', 'Min_elevation_time', 'Max_elevation_time', 't1_azimuth', 't2_azimuth', 't3_azimuth', 't4_azimuth', 'Min_azimuth',	'Max_azimuth', 'Min_azimuth_time', 'Max_azimuth_time', 'Overall_Min_Time', 'Overall_Max_Time'])

with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    results_combine_df.to_excel(writer, sheet_name="Observer_Building_Results", index=False) 

### RAM ISSUE SOLUTION

In [ ]:
# # Split the DataFrame into two equal parts
# df_split_1 = observer_obstacle_df.iloc[:len(observer_obstacle_df)//2]
# df_split_2 = observer_obstacle_df.iloc[len(observer_obstacle_df)//2:]

In [ ]:
# observer_building_combine_1 = []

# for idx, row in df_split_1.iterrows():
#     observer_wall = [
#         (row['Observer_Point1'][0], row['Observer_Point1'][1], row['Observer_Point1'][2]),
#         (row['Observer_Point3'][0], row['Observer_Point3'][1], row['Observer_Point3'][2]),
#         (row['Observer_Point2'][0], row['Observer_Point2'][1], row['Observer_Point2'][2]),
#         (row['Observer_Point4'][0], row['Observer_Point4'][1], row['Observer_Point4'][2]),
#         row['Observer_Building'], row['Observer_Tower'], row['Observer_Face']
#     ]
    
#     building_wall = [
#         (row['Obstacle_Point1'][0], row['Obstacle_Point1'][1], row['Obstacle_Point1'][2]),
#         (row['Obstacle_Point3'][0], row['Obstacle_Point3'][1], row['Obstacle_Point3'][2]),
#         (row['Obstacle_Point2'][0], row['Obstacle_Point2'][1], row['Obstacle_Point2'][2]),
#         (row['Obstacle_Point4'][0], row['Obstacle_Point4'][1], row['Obstacle_Point4'][2]),
#         row['Obstacle_Tower'], row['Obstacle_Face'], row['Obstacle_Building']
#     ]
    
#     building_status = check_buildings(building_wall[0][0], building_wall[0][1], building_wall[2][0], building_wall[2][1], observer_wall[0][0], observer_wall[0][1], sun_azimuth_morning, sun_azimuth_evening) and check_buildings(building_wall[0][0], building_wall[0][1], building_wall[2][0], building_wall[2][1], observer_wall[2][0], observer_wall[2][1], sun_azimuth_morning, sun_azimuth_evening)
    
#     #Plot sun_morning, sun_afternoon and sun_evening quivers in each plot
#     plot_shadow_analysis(observer_wall[0][0], observer_wall[0][1], observer_wall[2][0], observer_wall[2][1],
#                          building_wall[0][0], building_wall[0][1], building_wall[2][0], building_wall[2][1],
#                          sun_azimuth_morning, sun_azimuth_afternoon, sun_azimuth_evening, 
#                          observer_wall[5], observer_wall[6], building_wall[4], building_wall[5], building_wall[6])
        
#     cross_product_result = calculate_cross_product(observer_wall[:4], building_wall[:4])
    
#     wall_orientation_status = wall_orientation(cross_product_result[0][2], cross_product_result[1][2])

#     #Plot the observer tower as whole as well
#     plot_wall_orientations(observer_wall[0][0], observer_wall[0][1], observer_wall[2][0], observer_wall[2][1],
#                          building_wall[0][0], building_wall[0][1], building_wall[2][0], building_wall[2][1],
#                          observer_wall[4], observer_wall[5], observer_wall[6], building_wall[4], building_wall[5], building_wall[6])
            
#     if(wall_orientation_status != 'Back' and building_status != False):
#         ### ELEVATION FUNCTION CALCULATIONS
#         shadow_status_obs1_elevation, shadow_status_obs2_elevation, elevation, t1_angle_elevation, t2_angle_elevation, t3_angle_elevation, t4_angle_elevation = get_shadow_status_and_elevation(observer_wall[0][0], observer_wall[0][1], observer_wall[0][2], observer_wall[2][0], observer_wall[2][1], observer_wall[2][2],
#         building_wall[1][0], building_wall[1][1], building_wall[1][2], building_wall[3][0], building_wall[3][1], building_wall[3][2],
#         latitude, longitude, time)
#         #print([shadow_status_obs1_elevation, shadow_status_obs2_elevation])

#         max_elev = max(t1_angle_elevation, t2_angle_elevation, t3_angle_elevation, t4_angle_elevation)

#         closest_time_before_12, closest_time_after_12 = get_closest_times(max_elev, solar_file)

#         ### AZIMUTH FUNCTION CALCULATION
#         shadow_status_1_azimuth, shadow_status_2_azimuth, azimuth, t1_angle_azimuth, t2_angle_azimuth, t3_angle_azimuth, t4_angle_azimuth = get_shadow_status_and_azimuth(
#             observer_wall[0][0], observer_wall[0][1], observer_wall[2][0], observer_wall[2][1], building_wall[1][0], building_wall[1][1], building_wall[3][0], building_wall[3][1], latitude, longitude, time)
#         #print([shadow_status_1_azimuth, shadow_status_2_azimuth])

#         min_azimuth = min(t1_angle_azimuth, t2_angle_azimuth, t3_angle_azimuth, t4_angle_azimuth)
#         max_azimuth = max(t1_angle_azimuth, t2_angle_azimuth, t3_angle_azimuth, t4_angle_azimuth)

#         min_azimuth_time = get_time_from_azimuth(min_azimuth, solar_file)
#         max_azimuth_time = get_time_from_azimuth(max_azimuth, solar_file)

#         # min_time should be lower bound of overlap of both azimuth and elevation times. If no overlap, return no shadow
#         min_time = max(closest_time_before_12, min_azimuth_time)
#         max_time = min(closest_time_after_12, max_azimuth_time)

#     else:
#         t1_angle_elevation, t2_angle_elevation, t3_angle_elevation, t4_angle_elevation, max_elev, closest_time_after_12, closest_time_before_12, t1_angle_azimuth, t2_angle_azimuth, t3_angle_azimuth, t4_angle_azimuth, min_azimuth, max_azimuth, min_azimuth_time, max_azimuth_time, min_time, max_time = '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'
        
#     observer_building_combine_1.append([
#         observer_wall[4],                               # Observer tower number
#         observer_wall[5],                               # Observer wall number
#         observer_wall[0:4],                             # Coordinates of the observer wall
#         building_wall[6],                               # Building number
#         building_wall[4],                               # Building tower number
#         building_wall[5],                               # Building wall number
#         building_wall[0:4],                             # Coordinates of the building wall
#         building_status,                                # Status of shadow casting
#         cross_product_result[0][2],                     # z-component of the first cross product
#         cross_product_result[1][2],                     # z-component of the second cross product
#         wall_orientation_status,
#         t1_angle_elevation,
#         t2_angle_elevation,
#         t3_angle_elevation,
#         t4_angle_elevation,
#         max_elev,
#         closest_time_before_12,
#         closest_time_after_12,
#         t1_angle_azimuth,
#         t2_angle_azimuth,
#         t3_angle_azimuth,
#         t4_angle_azimuth,
#         min_azimuth,
#         max_azimuth,
#         min_azimuth_time,
#         max_azimuth_time,
#         min_time,
#         max_time
#         ])

# # Convert to DataFrame
# results_combine_df_1 = pd.DataFrame(observer_building_combine_1, columns=['Tower_number', 'Observer_wall_no', 'observer_walls', 'Building No', 'tower_no', 'Building_wall_no', 'Building_walls', 'Building_status', 'ab*aw1_z', 'ab*aw2_z', 'wall_orientation', 't1_elevation', 't2_elevation', 't3_elevation', 't4_elevation', 'max_elevation', 'Min_elevation_time', 'Max_elevation_time', 't1_azimuth', 't2_azimuth', 't3_azimuth', 't4_azimuth', 'Min_azimuth',	'Max_azimuth', 'Min_azimuth_time', 'Max_azimuth_time', 'Overall_Min_Time', 'Overall_Max_Time'])


Successfully saved shadow plot: C:\Users\Kalpesh\Downloads\Jayesh Testing\Final_Output\Plots\North_wall_new\T1_1_B1_T1_2.png
Successfully saved shadow plot: C:\Users\Kalpesh\Downloads\Jayesh Testing\Final_Output\Plots\North_wall_new\T1_1_B1_T1_3.png
Successfully saved shadow plot: C:\Users\Kalpesh\Downloads\Jayesh Testing\Final_Output\Plots\North_wall_new\T1_1_B1_T1_4.png
Successfully saved shadow plot: C:\Users\Kalpesh\Downloads\Jayesh Testing\Final_Output\Plots\North_wall_new\T1_2_B1_T1_1.png
Successfully saved shadow plot: C:\Users\Kalpesh\Downloads\Jayesh Testing\Final_Output\Plots\North_wall_new\T1_2_B1_T1_3.png
Successfully saved shadow plot: C:\Users\Kalpesh\Downloads\Jayesh Testing\Final_Output\Plots\North_wall_new\T1_2_B1_T1_4.png
Successfully saved shadow plot: C:\Users\Kalpesh\Downloads\Jayesh Testing\Final_Output\Plots\North_wall_new\T1_3_B1_T1_1.png
Successfully saved shadow plot: C:\Users\Kalpesh\Downloads\Jayesh Testing\Final_Output\Plots\North_wall_new\T1_3_B1_T1_2.png


In [ ]:
# observer_building_combine_2 = []

# for idx, row in df_split_2.iterrows():
#     observer_wall = [
#         (row['Observer_Point1'][0], row['Observer_Point1'][1], row['Observer_Point1'][2]),
#         (row['Observer_Point3'][0], row['Observer_Point3'][1], row['Observer_Point3'][2]),
#         (row['Observer_Point2'][0], row['Observer_Point2'][1], row['Observer_Point2'][2]),
#         (row['Observer_Point4'][0], row['Observer_Point4'][1], row['Observer_Point4'][2]),
#         row['Observer_Building'], row['Observer_Tower'], row['Observer_Face']
#     ]
    
#     building_wall = [
#         (row['Obstacle_Point1'][0], row['Obstacle_Point1'][1], row['Obstacle_Point1'][2]),
#         (row['Obstacle_Point3'][0], row['Obstacle_Point3'][1], row['Obstacle_Point3'][2]),
#         (row['Obstacle_Point2'][0], row['Obstacle_Point2'][1], row['Obstacle_Point2'][2]),
#         (row['Obstacle_Point4'][0], row['Obstacle_Point4'][1], row['Obstacle_Point4'][2]),
#         row['Obstacle_Tower'], row['Obstacle_Face'], row['Obstacle_Building']
#     ]
    
#     building_status = check_buildings(building_wall[0][0], building_wall[0][1], building_wall[2][0], building_wall[2][1], observer_wall[0][0], observer_wall[0][1], sun_azimuth_morning, sun_azimuth_evening) and check_buildings(building_wall[0][0], building_wall[0][1], building_wall[2][0], building_wall[2][1], observer_wall[2][0], observer_wall[2][1], sun_azimuth_morning, sun_azimuth_evening)
    
#     #Plot sun_morning, sun_afternoon and sun_evening quivers in each plot
#     plot_shadow_analysis(observer_wall[0][0], observer_wall[0][1], observer_wall[2][0], observer_wall[2][1],
#                          building_wall[0][0], building_wall[0][1], building_wall[2][0], building_wall[2][1],
#                          sun_azimuth_morning, sun_azimuth_afternoon, sun_azimuth_evening, 
#                          observer_wall[5], observer_wall[6], building_wall[4], building_wall[5], building_wall[6])
        
#     cross_product_result = calculate_cross_product(observer_wall[:4], building_wall[:4])
    
#     wall_orientation_status = wall_orientation(cross_product_result[0][2], cross_product_result[1][2])

#     #Plot the observer tower as whole as well
#     plot_wall_orientations(observer_wall[0][0], observer_wall[0][1], observer_wall[2][0], observer_wall[2][1],
#                          building_wall[0][0], building_wall[0][1], building_wall[2][0], building_wall[2][1],
#                          observer_wall[4], observer_wall[5], observer_wall[6], building_wall[4], building_wall[5], building_wall[6])
            
#     if(wall_orientation_status != 'Back' and building_status != False):
#         ### ELEVATION FUNCTION CALCULATIONS
#         shadow_status_obs1_elevation, shadow_status_obs2_elevation, elevation, t1_angle_elevation, t2_angle_elevation, t3_angle_elevation, t4_angle_elevation = get_shadow_status_and_elevation(observer_wall[0][0], observer_wall[0][1], observer_wall[0][2], observer_wall[2][0], observer_wall[2][1], observer_wall[2][2],
#         building_wall[1][0], building_wall[1][1], building_wall[1][2], building_wall[3][0], building_wall[3][1], building_wall[3][2],
#         latitude, longitude, time)
#         #print([shadow_status_obs1_elevation, shadow_status_obs2_elevation])

#         max_elev = max(t1_angle_elevation, t2_angle_elevation, t3_angle_elevation, t4_angle_elevation)

#         closest_time_before_12, closest_time_after_12 = get_closest_times(max_elev, solar_file)

#         ### AZIMUTH FUNCTION CALCULATION
#         shadow_status_1_azimuth, shadow_status_2_azimuth, azimuth, t1_angle_azimuth, t2_angle_azimuth, t3_angle_azimuth, t4_angle_azimuth = get_shadow_status_and_azimuth(
#             observer_wall[0][0], observer_wall[0][1], observer_wall[2][0], observer_wall[2][1], building_wall[1][0], building_wall[1][1], building_wall[3][0], building_wall[3][1], latitude, longitude, time)
#         #print([shadow_status_1_azimuth, shadow_status_2_azimuth])

#         min_azimuth = min(t1_angle_azimuth, t2_angle_azimuth, t3_angle_azimuth, t4_angle_azimuth)
#         max_azimuth = max(t1_angle_azimuth, t2_angle_azimuth, t3_angle_azimuth, t4_angle_azimuth)

#         min_azimuth_time = get_time_from_azimuth(min_azimuth, solar_file)
#         max_azimuth_time = get_time_from_azimuth(max_azimuth, solar_file)

#         # min_time should be lower bound of overlap of both azimuth and elevation times. If no overlap, return no shadow
#         min_time = max(closest_time_before_12, min_azimuth_time)
#         max_time = min(closest_time_after_12, max_azimuth_time)

#     else:
#         t1_angle_elevation, t2_angle_elevation, t3_angle_elevation, t4_angle_elevation, max_elev, closest_time_after_12, closest_time_before_12, t1_angle_azimuth, t2_angle_azimuth, t3_angle_azimuth, t4_angle_azimuth, min_azimuth, max_azimuth, min_azimuth_time, max_azimuth_time, min_time, max_time = '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'
#     observer_building_combine_2.append([
#         observer_wall[4],                               # Observer tower number
#         observer_wall[5],                               # Observer wall number
#         observer_wall[0:4],                             # Coordinates of the observer wall
#         building_wall[6],                               # Building number
#         building_wall[4],                               # Building tower number
#         building_wall[5],                               # Building wall number
#         building_wall[0:4],                             # Coordinates of the building wall
#         building_status,                                # Status of shadow casting
#         cross_product_result[0][2],                     # z-component of the first cross product
#         cross_product_result[1][2],                     # z-component of the second cross product
#         wall_orientation_status,
#         t1_angle_elevation,
#         t2_angle_elevation,
#         t3_angle_elevation,
#         t4_angle_elevation,
#         max_elev,
#         closest_time_before_12,
#         closest_time_after_12,
#         t1_angle_azimuth,
#         t2_angle_azimuth,
#         t3_angle_azimuth,
#         t4_angle_azimuth,
#         min_azimuth,
#         max_azimuth,
#         min_azimuth_time,
#         max_azimuth_time,
#         min_time,
#         max_time
#         ])

# # Convert to DataFrame
# results_combine_df_2 = pd.DataFrame(observer_building_combine_2, columns=['Tower_number', 'Observer_wall_no', 'observer_walls', 'Building No', 'tower_no', 'Building_wall_no', 'Building_walls', 'Building_status', 'ab*aw1_z', 'ab*aw2_z', 'wall_orientation', 't1_elevation', 't2_elevation', 't3_elevation', 't4_elevation', 'max_elevation', 'Min_elevation_time', 'Max_elevation_time', 't1_azimuth', 't2_azimuth', 't3_azimuth', 't4_azimuth', 'Min_azimuth',	'Max_azimuth', 'Min_azimuth_time', 'Max_azimuth_time', 'Overall_Min_Time', 'Overall_Max_Time'])


Successfully saved shadow plot: C:\Users\Kalpesh\Downloads\Jayesh Testing\Final_Output\Plots\North_wall_new\T8_1_B1_T1_2.png
Successfully saved shadow plot: C:\Users\Kalpesh\Downloads\Jayesh Testing\Final_Output\Plots\North_wall_new\T8_1_B1_T1_3.png
Successfully saved shadow plot: C:\Users\Kalpesh\Downloads\Jayesh Testing\Final_Output\Plots\North_wall_new\T8_1_B1_T1_4.png
Successfully saved shadow plot: C:\Users\Kalpesh\Downloads\Jayesh Testing\Final_Output\Plots\North_wall_new\T8_2_B1_T1_1.png
Successfully saved shadow plot: C:\Users\Kalpesh\Downloads\Jayesh Testing\Final_Output\Plots\North_wall_new\T8_2_B1_T1_3.png
Successfully saved shadow plot: C:\Users\Kalpesh\Downloads\Jayesh Testing\Final_Output\Plots\North_wall_new\T8_2_B1_T1_4.png
Successfully saved shadow plot: C:\Users\Kalpesh\Downloads\Jayesh Testing\Final_Output\Plots\North_wall_new\T8_3_B1_T1_1.png
Successfully saved shadow plot: C:\Users\Kalpesh\Downloads\Jayesh Testing\Final_Output\Plots\North_wall_new\T8_3_B1_T1_2.png


In [ ]:
# # Merge the two dataframes
# final_results_combine_df = pd.concat([results_combine_df_1, results_combine_df_2], ignore_index=True)

# # Write the merged dataframe to Excel
# with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
#     final_results_combine_df.to_excel(writer, sheet_name="Observer_Building_Results", index=False)